In [3]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
import operator

# ==============================================================================
# 2. DEFINE THE STATE AND WORKFLOW LOGIC
# ==============================================================================
# The key to managing memory:
# Using Annotated with operator.add tells LangGraph to append to the 'messages' list
# instead of overwriting it. This builds the conversation history.
class ChatState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

def chatbot_node(state: ChatState) -> dict:
    last_message = state["messages"][-1].content
    print(f"  User said: '{last_message}'")

    response = f"I acknowledge your message: '{last_message}'. How can I help further?"
    print(f"  AI says: '{response}'")

    return {"messages": [AIMessage(content=response)]}

# ==============================================================================
# 3. BUILD AND COMPILE THE WORKFLOW
# ==============================================================================
def create_chatbot_workflow():
    workflow = StateGraph(ChatState)
    workflow.add_node("chatbot", chatbot_node)
    workflow.add_edge(START, "chatbot")
    workflow.add_edge("chatbot", END)

    memory = SqliteSaver.from_conn_string(":memory:")
    return workflow.compile(checkpointer=memory), memory

# ==============================================================================
# 4. RUN THE WORKFLOW AND DEMONSTRATE MEMORY
# ==============================================================================
if __name__ == "__main__":
    app, memory = create_chatbot_workflow()

    # ✅ FIX: wrap thread_id inside configurable
    thread_config = {"configurable": {"thread_id": "chat-session-1"}}

    print("--- Turn 1 ---")
    app.invoke({"messages": [HumanMessage(content="Hello, is anyone there?")]}, config=thread_config)

    print("\n--- Turn 2 ---")
    app.invoke({"messages": [HumanMessage(content="I need help with my account.")]}, config=thread_config)

    print("\n--- Turn 3 ---")
    app.invoke({"messages": [HumanMessage(content="The issue is with my password.")]}, config=thread_config)

    print("\n" + "="*50)
    print("--- Retrieving full conversation history from the checkpointer ---")

    # ✅ FIX: get state using proper thread_config
    final_state = app.get_state(config=thread_config)

    for message in final_state.values["messages"]:
        print(f"  [{message.type.upper()}] {message.content}")


--- Turn 1 ---
  User said: 'Hello, is anyone there?'
  AI says: 'I acknowledge your message: 'Hello, is anyone there?'. How can I help further?'

--- Turn 2 ---
  User said: 'I need help with my account.'
  AI says: 'I acknowledge your message: 'I need help with my account.'. How can I help further?'

--- Turn 3 ---
  User said: 'The issue is with my password.'
  AI says: 'I acknowledge your message: 'The issue is with my password.'. How can I help further?'

--- Retrieving full conversation history from the checkpointer ---
  [HUMAN] Hello, is anyone there?
  [AI] I acknowledge your message: 'Hello, is anyone there?'. How can I help further?
  [HUMAN] I need help with my account.
  [AI] I acknowledge your message: 'I need help with my account.'. How can I help further?
  [HUMAN] The issue is with my password.
  [AI] I acknowledge your message: 'The issue is with my password.'. How can I help further?
